In [2]:
import os
# import sys

# from src.processing import mean_pooling, mean_pooling_embedding_with_normalization

# sys.path.append('.')

import argparse
import json
import numpy as np

import faiss
import torch
# from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

/Users/amarkanaka/miniconda3/envs/faiss_1.8.0_new_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Embeddings Model is a mapping from chunks of max length 512 tokens -> R^768 ???
dim = 768
#normalize embeddings
norm = True
dataset = 'musique'
# unit = 'proposition'

In [4]:
model_label = 'facebook_contriever'

In [5]:
vector_path = f'data/{dataset}/{dataset}_{model_label}_proposition_vectors_norm.npy'
index_path = f'data/{dataset}/{dataset}_{model_label}_proposition_ip_norm.index'
if(os.path.isfile(vector_path)):
    vectors = np.load(vector_path)

In [6]:
vector_path

'data/musique/musique_facebook_contriever_proposition_vectors_norm.npy'

In [7]:
if dataset == 'musique':
    corpus = json.load(open('data/musique_proposition_corpus.json', 'r'))
elif dataset == '2wikimultihopqa':
    corpus = json.load(open('data/2wikimultihopqa_proposition_corpus.json', 'r'))
corpus_contents = []
for item in corpus:
    corpus_contents.append(item['title'] + '\n' + item['propositions'])
print('corpus size: {}'.format(len(corpus_contents)))

corpus size: 11656


In [8]:
total_len = 0
max_len = 0
min_len = 1000000
for line in corpus_contents:
    total_len += len(line)
    if len(line) > max_len:
        max_len = len(line)
    if len(line) < min_len:
        min_len = len(line)
print(max_len / 4)
print(min_len / 4)
print((total_len / len(corpus_contents)) / 4)    

525.25
31.25
149.56335792724778


In [14]:
if os.path.isfile(vector_path):
    print('Loading existing vectors:', vector_path)
    vectors = np.load(vector_path)
    print('Vectors loaded:', len(vectors))
else:
    # load model
    tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
    model = AutoModel.from_pretrained('facebook/contriever')
    # Check if multiple GPUs are available and if so, use them all
    if not torch.backends.mps.is_available():
        if not torch.backends.mps.is_built():
            print("MPS not available because the current PyTorch install was not "
                "built with MPS enabled.")
        else:
            print("MPS not available because the current MacOS version is not 12.3+ "
                "and/or you do not have an MPS-enabled device on this machine.")
    else:
        # print("device available")
        mps_device = torch.device("mps")    
        model.to(mps_device)
        model = torch.nn.DataParallel(model)
    #test batch size = 16 and batch size = 32 
    batch_size = 16
    vectors = np.zeros((len(corpus_contents), dim))
    #get batch_size # of entries from corpus_contents, tokenize and embed them in 768 dimensional space
    # for idx in range(0, len(corpus_contents), batch_size):
    #     end_idx = min(idx + batch_size, len(corpus_contents))
    #     seqs = corpus_contents[idx:end_idx]
        # tokens = tokenizer()

In [24]:
corpus_contents[0]

"Lionel Messi\nLionel Messi spent a year at Barcelona's youth academy, La Masia. Messi was enrolled in the Royal Spanish Football Federation (RFEF) in February 2002. Messi played in all competitions. Messi befriended his teammates, Cesc Fàbregas and Gerard Piqué. Cesc Fàbregas and Gerard Piqué were among Lionel Messi's teammates. Messi completed his growth hormone treatment aged 14. Messi became an integral part of the Baby Dream Team, Barcelona's greatest-ever youth side. During his first full season (2002-03), Messi was top scorer with 36 goals in 30 games for the Cadetes A. The Cadetes A won an unprecedented treble of the league and both the Spanish and Catalan cups. The Copa Catalunya final was a 4-1 victory over Espanyol. The Copa Catalunya final became known in club lore as the partido de la máscara, the final of the mask. Messi suffered a broken cheekbone during a league match. Messi was allowed to start the game on the condition that he wear a plastic protector. Messi was hinde

In [27]:
corpus_contents[1]

"FC Barcelona\nFC Barcelona was the favourites and started strongly. Barcelona finished the 2006-07 season without trophies. A pre-season US tour was blamed for a string of injuries to key players. Leading scorer Eto'o was injured during the US tour. Rising star Lionel Messi was injured during the US tour. There was open feuding between Eto'o and coach Frank Rijkaard and Ronaldinho. Ronaldinho admitted that a lack of fitness affected his form. In La Liga, Barcelona were in first place for much of the season. Inconsistency in the New Year saw Real Madrid overtake Barcelona to become champions. Barcelona advanced to the semi-finals of the Copa del Rey. Barcelona won the first leg against Getafe 5-2. A goal from Messi brought comparison to Diego Maradona's goal of the century. Barcelona lost the second leg 4-1. Barcelona took part in the 2006 FIFA Club World Cup. Barcelona was beaten by a late goal in the final against Brazilian side Internacional. In the Champions League, Barcelona were 

In [25]:
#longer sequence
len(corpus_contents[0])

1339

In [26]:
#shorter sequence
len(corpus_contents[1])

1091

In [32]:
encoding_ex = tokenizer(corpus_contents[:2],padding = True, truncation= True, return_tensors='pt').to(mps_device)

In [ ]:
encoding_ex["input_ids"][0]

In [35]:
outputs = model(**encoding_ex)
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0193, -0.2598, -0.0947,  ...,  0.1957, -0.2767, -0.0861],
         [-0.2650,  0.2685,  0.2683,  ...,  0.0458,  0.0605, -0.0593],
         [-0.0595, -0.1592,  0.0049,  ..., -0.0088,  0.0349,  0.0666],
         ...,
         [-0.1642,  0.3410,  0.1903,  ..., -0.1007, -0.2093, -0.4115],
         [ 0.1654, -0.0760,  0.0687,  ...,  0.1888, -0.0068,  0.1188],
         [ 0.0531, -0.2163,  0.0448,  ...,  0.1647, -0.0648, -0.0252]],

        [[ 0.1521, -0.1501, -0.1869,  ..., -0.1612, -0.0992, -0.0393],
         [-0.5969, -0.1441,  0.2693,  ..., -0.3138,  0.1886, -0.6843],
         [-0.0079,  0.3074,  0.1848,  ..., -0.0957, -0.0184, -0.3032],
         ...,
         [ 0.2553, -1.0877,  0.1557,  ...,  0.3257,  0.0293,  0.0979],
         [ 0.5607, -0.9700,  0.0770,  ...,  0.2181,  0.0424, -0.0582],
         [ 0.6028, -1.2719,  0.0332,  ...,  0.3162,  0.0034,  0.0091]]],
       device='mps:0', grad_fn=<NativeLayerNormBackwa

Originally, we have 315 token IDS to describe our tokenization of the first line in the corpus and the second line in the corpus. We then map each token to a 768 dimensional vector. The dimension increases because we take a "bag of words approach" and create token-level embeddings

In [38]:
outputs[0].shape

torch.Size([2, 315, 768])

In [77]:
outputs[0]

tensor([[[-0.0193, -0.2598, -0.0947,  ...,  0.1957, -0.2767, -0.0861],
         [-0.2650,  0.2685,  0.2683,  ...,  0.0458,  0.0605, -0.0593],
         [-0.0595, -0.1592,  0.0049,  ..., -0.0088,  0.0349,  0.0666],
         ...,
         [-0.1642,  0.3410,  0.1903,  ..., -0.1007, -0.2093, -0.4115],
         [ 0.1654, -0.0760,  0.0687,  ...,  0.1888, -0.0068,  0.1188],
         [ 0.0531, -0.2163,  0.0448,  ...,  0.1647, -0.0648, -0.0252]],

        [[ 0.1521, -0.1501, -0.1869,  ..., -0.1612, -0.0992, -0.0393],
         [-0.5969, -0.1441,  0.2693,  ..., -0.3138,  0.1886, -0.6843],
         [-0.0079,  0.3074,  0.1848,  ..., -0.0957, -0.0184, -0.3032],
         ...,
         [ 0.2553, -1.0877,  0.1557,  ...,  0.3257,  0.0293,  0.0979],
         [ 0.5607, -0.9700,  0.0770,  ...,  0.2181,  0.0424, -0.0582],
         [ 0.6028, -1.2719,  0.0332,  ...,  0.3162,  0.0034,  0.0091]]],
       device='mps:0', grad_fn=<NativeLayerNormBackward0>)

In [97]:
outputs[0].masked_fill(~encoding_ex["attention_mask"][...,None].bool(),0)

tensor([[[-0.0193, -0.2598, -0.0947,  ...,  0.1957, -0.2767, -0.0861],
         [-0.2650,  0.2685,  0.2683,  ...,  0.0458,  0.0605, -0.0593],
         [-0.0595, -0.1592,  0.0049,  ..., -0.0088,  0.0349,  0.0666],
         ...,
         [-0.1642,  0.3410,  0.1903,  ..., -0.1007, -0.2093, -0.4115],
         [ 0.1654, -0.0760,  0.0687,  ...,  0.1888, -0.0068,  0.1188],
         [ 0.0531, -0.2163,  0.0448,  ...,  0.1647, -0.0648, -0.0252]],

        [[ 0.1521, -0.1501, -0.1869,  ..., -0.1612, -0.0992, -0.0393],
         [-0.5969, -0.1441,  0.2693,  ..., -0.3138,  0.1886, -0.6843],
         [-0.0079,  0.3074,  0.1848,  ..., -0.0957, -0.0184, -0.3032],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       device='mps:0', grad_fn=<MaskedFillBackward0>)

In [106]:
outputs[0].sum(dim=1)/encoding_ex["attention_mask"].sum(dim=1).unsqueeze(-1)

tensor([[-0.0237, -0.0938,  0.0261,  ...,  0.0126, -0.0239, -0.0051],
        [ 0.2324, -0.2980, -0.0321,  ..., -0.0242, -0.0004, -0.0803]],
       device='mps:0', grad_fn=<DivBackward0>)